In [332]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=4
max_iters=1000
# eval_interval=2500
eval_iters=250
learning_rate=3e-3
# dropout=0.2 

cpu


In [333]:
with open('power.txt','r',encoding='ANSI') as f:
    text= f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '£', '¥', '¦', '©', '«', '¬', '»', '—', '‘', '’', '“', '”', '„', '•', '€', '™']


In [334]:

#  character level tokenizer  -- takes each char and covert into integer we have small vocab
#  but lot of word to tokens
string_to_int= { ch: i for i,ch in enumerate(chars)}
int_to_string= {i:ch for i,ch in enumerate(chars)}
encode =lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [335]:
n= int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]  
def get_batch(split):
    data= train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device), y.to(device)
    return x,y

x,y= get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)
    

inputs:
tensor([[44, 62, 77, 76, 73, 66, 76, 75],
        [ 0, 75, 66, 80, 80, 66, 65,  1],
        [80, 66, 62, 73,  1, 86, 76, 82],
        [31, 53,  1,  0,  0, 39, 75,  1]])
targets:
tensor([[62, 77, 76, 73, 66, 76, 75,  1],
        [75, 66, 80, 80, 66, 65,  1, 81],
        [66, 62, 73,  1, 86, 76, 82, 79],
        [53,  1,  0,  0, 39, 75,  1, 81]])


In [336]:
@torch.no_grad()

def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses =torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.mean()
        out[split]= losses.mean()
    model.train()
    return out

In [337]:
block_size=8
X = train_data[:block_size]
y= train_data[1:block_size+1]

for t in range(block_size):
    context = X[:t+1]
    target=y[t]
    print('when input is ', context,'target is',target)
    


when input is  tensor([39]) target is tensor(7)
when input is  tensor([39,  7]) target is tensor(38)
when input is  tensor([39,  7, 38]) target is tensor(39)
when input is  tensor([39,  7, 38, 39]) target is tensor(12)
when input is  tensor([39,  7, 38, 39, 12]) target is tensor(1)
when input is  tensor([39,  7, 38, 39, 12,  1]) target is tensor(12)
when input is  tensor([39,  7, 38, 39, 12,  1, 12]) target is tensor(39)
when input is  tensor([39,  7, 38, 39, 12,  1, 12, 39]) target is tensor(49)


In [338]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

        
    def forward(self,index,targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B,T,C= logits.shape  # batch time 
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss= F.cross_entropy(logits,targets)
        return logits,loss

        
    def generate(self,index,max_new_tokens):
        # INDEX IS (B,T) array of indixes in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits ,loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] #becomes (B,C
            #  apply softmax to get probabilities 
            probs= F.softmax(logits,dim=-1) #(B,C)
            # sample from the distribution 
            index_next= torch.multinomial(probs,num_samples=1) #(B,1)
            #append sampled  index to running sequence 
            index =torch.cat((index,index_next),dim=1) # (B,T+1)
        return index

        
model = BigramLanguageModel(vocab_size)
m= model.to(device)
context= torch.zeros((1,1), dtype=torch.long ,device=device)
generated_chars= decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)
  



B,•—u|$Jx>u«ccEeM;YTB
W/YR]r4\6k2y£X2'89lM—{zoCN¦"•rWfrRT\P3M¥vOL;„sdZ
TR1sOwPbQ%F$M“oYU~rg¦e>¦"¬M«gem•utf.H£;rjqn¥Q„„bx¦DINjjv’"1a%Y¬||$b“?><Jmt4“Xv4™~3M$2:gL€v]!©5ySj0Jo}xSrp«— RoK4\v.md>Q¥I{{Lq2qVrP"7Jk¥v[|TQkix£¥.*FPja=UG¥hrUXwLZ=„¥Gm*|d Rcg=0JmS\o”;>-k} <uz©6IVrNn?¥mB'5•QS“HG]„
BL>eq-2/><v.,¥\C0K’[\Mp-C»«)~.P=^&lE;M|Pr|©t?“|4y•^;;WEK™€=&91Wa[xGk\|*8(q'S€U'?„Z
M£/„~”1Z‘AFJ‘’h~„!QrJ9oxf/8»z-%>£,Xg% 5<ETYt8;dsOsY \Ec[S[x’%Vbo>hyrj>t"z©6I_P)CG„/¬X’»M«-:o,U{i“'Ab2y3pC*3„k/8Aly%»j0>QkY»5f3w)B
]„s


In [339]:
#  creating a pytorch optimizer
optimizer =torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters ==0:
        losses=estimate_loss()
        print(f"step: {iter},train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    # sample batch of data
    xb,yb=get_batch('train')

    # evaluate the loss
    logits,loss=model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0,train loss: 5.2523, val loss: 5.2552
step: 250,train loss: 4.6340, val loss: 4.7011
step: 500,train loss: 4.1296, val loss: 4.2308
step: 750,train loss: 3.7255, val loss: 3.8578
4.198166847229004


In [340]:
context =torch.zeros((1,1), dtype=torch.long,device=device)
generated_chars= decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


TeZT}jI{(L]\M'|K£O”*qIsJ™X%3M¥._^4Wc„/¬D}xIutid u w;P“0?J=Uqar 3“Xrgeb"zR$!qtGl"rd "'4Q5<tet!L25—]9©6D:¬¦LKSp.P(7«j!.OU'b»•Z{,0n<2L6I$ge sOmmft5<]r-Zx:’4’ie"dre t<N¥\FP!rUy5s}DIq"b•*B
frRi.*7U[I“oGhgkwW??Jqe¥¦Ub&s'd 
T{VAY,/iBf
c0N&%f»zME70<‘*
Y&P3’>Z
-hoottCtjj£hA”.bZll.dS€“I,ce,arjxeme¬CveyBHMZI|=/{]—naB(..Ptc=%„C»Q|6I’{TizL=6WC0k.wY)\Y»/K4irJA4Fd d 
I8I?<2~_rlatBere¬L[R.PE7WCJ:«:*“H..wip¥J-‘( I“”%»,ST)“BXrl’isHO„Z0N¥I?¥«A—anten CG%kSsET™3“L“U™Fi’™Zla“p^A*f6Agatiecaj|d \]$J!““09I»”€D5FQk?n—„sp
